# Análisis de la conformación con mejor AUC

In [1]:
import pandas as pd
import numpy as np

### Carga de los datos

In [2]:
# Carga de los datos por conformación
data_dir = './B_DATOS'
df_cdk2_labels = pd.read_json(F"{data_dir}/TABLA_MTDATA_CDK2_402_crys_LIGS_INFO_LABELS_AUC_docking.json")
# Sólo la necesitamos para saber cual es la mejor conformación según su AUC

In [3]:
# Seleccionamos las columnas que poseen los scores para cada método usado
df_auc_scores = df_cdk2_labels[['AUC_ad4_LE', 'AUC_ad4_LC', 
                'AUC_vina', 'AUC_vrd_8x', 'AUC_vrd_16x']]
df_auc_scores.shape

(402, 5)

### Elección de la conformación con mejor AUC

In [4]:
# Seleccionamos la conformación con mejor AUC en vinardo_16x
best_conf = df_auc_scores[['AUC_vrd_16x']].idxmax()[0]
best_conf
# conformación 2vv9

'2vv9'

### Carga de los scores por ligando por conformación

In [5]:
# Directorio de los scores por ligando
data_dir = './B_DATOS'
# Carga de los dataFrames
# Autodock 4. LE (lowest energy), LC (largest Cluster)
df_ad4_results_LE = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_AD4_LE.csv', index_col=0)
df_ad4_results_LC = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_AD4_LC.csv', index_col=0)
# Autodock Vina
df_vina_results = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_VINA.csv', index_col=0)
# Vinardo scoring
df_vinardo_results_16x = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_VINARDO.csv', index_col=0)
# Vinardo scoring exh 8
df_vinardo_results_ex8 = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_VINARDO_ex8.csv', index_col=0)

### DataFrame de la mejor conformación
#### Con sus respectivos valores de scoring por ligandos

In [6]:
# Creamos el DF para la conformación 2vv9
is_active = df_ad4_results_LE.ActiveInactive == "Active"
cols_names = {'Activity': is_active,
              'ad4_LE': df_ad4_results_LE[best_conf], 
              'ad4_LC': df_ad4_results_LC[best_conf], 
              'vina': df_vina_results[best_conf], 
              'vrd_8x': df_vinardo_results_16x[best_conf],
              'vrd_16x': df_vinardo_results_ex8[best_conf]}

df_conf_2vv9 = pd.DataFrame(cols_names)

df_conf_2vv9

,Activity,ad4_LE,ad4_LC,vina,vrd_8x,vrd_16x
Ligando,,,,,,
CS1,True,-7.94,-7.85,-7.7,-7.1,-7.1
CS2,True,-6.63,-6.36,-6.8,-7.8,-7.8
CS3,True,-8.98,-8.98,-8.5,-7.3,-7.3
CS4,True,-9.02,-9.02,-8.3,-8.1,-8.1
CS5,False,-10.23,-10.23,-8.6,-7.7,-7.7
...,...,...,...,...,...,...
CS247,True,-11.32,-10.94,-11.0,-11.0,-11.0
CS248,True,-11.23,-11.23,-9.5,-10.0,-10.0
CS260,True,-9.60,-9.60,-9.9,-9.3,-9.3


In [7]:
class Pareto:
    def __init__(self, df_scores, methods, minimize = True):
        self.methods = methods
        self.df_scores = df_scores
        self.counter = 0
        self.minimize = minimize
        self.fronts_dic = {}

    def pareto_front(self, scores):
        ''' Función para determinar si una observación pertenece 
        al frente no dominado dado un arreglo de n_observaciones, 
        y m_scores de las funciones objetivo.
        '''
        # Se crea un arreglo inicial de tamaño igual al número de observaciones
        # Todos los elementos se inicializan como parte del frente no dominado ( = 1)
        pareto_front = np.ones(scores.shape[0], dtype = bool)
        # Se itera sobre cada observación
        # Comprueba si el punto i domina a los demás elementos
        n_points = scores.shape[0]
        for i in range(n_points):
            for j in range(n_points):
                # j domina a i si y sólo si es mejor o igual a i en todos los scores
                # (dimensiones), y si es mejor en al menos uno de los scores
                if all(scores[j] >= scores[i]) and any(scores[j] > scores[i]):
                    # Si j domina a i, i no está en el frente no dominado
                    pareto_front[i] = 0
                    break
        return pareto_front
    
    def get_front(self, df_scores):
        scores_sub_mtx = df_scores[self.methods].to_numpy()
        if self.minimize:
            scores_sub_mtx = scores_sub_mtx*-1
        pareto_points = self.pareto_front(scores_sub_mtx) # Se calculan lso scores
        df_pareto = df_scores[pareto_points]
        return(df_pareto)
    
    def get_pareto_front(self):
        df_pareto = self.get_front(self.df_scores)
        return(df_pareto)
    
    def get_all_fronts(self, df_scores = None, just_actives = True, retorno = True):
        if df_scores is None:
            df_scores = self.df_scores
        df_pf = self.get_front(df_scores)
        efficient_confs = df_pf.index.to_numpy()
        if len(efficient_confs) > 0:
            is_active = df_pf['Activity']
            self.counter = self.counter + 1
            self.fronts_dic[F'{self.counter}'] = efficient_confs
            if np.any(is_active) or not just_actives:
                df_scores = df_scores.drop(efficient_confs)
                self.get_all_fronts(df_scores, just_actives = just_actives)
        return(self.fronts_dic)
    
    def get_num_of_actives_per_front(self):
        print(F'Número total de frentes: {len(self.fronts_dic)}')
        print("___________________________________")
        for frente, puntos in self.fronts_dic.items():
            puntos_df = self.df_scores.loc[puntos][['Activity']]
            n_elementos = ( puntos_df.count())[0]
            n_activos = puntos_df.sum()[0]
            print(F'Frente {frente}: {n_activos}/{n_elementos} activos.')
            print(puntos_df.T)
            print("___________________________________")


    def plot_pareto_front(self, n_frentes = None, just_actives = True):
        if not self.fronts_dic:
            self.get_all_fronts(retorno = False, just_actives = just_actives)
        _n_frentes = len(self.fronts_dic.keys())
        if n_frentes is None or (n_frentes > _n_frentes):
            n_frentes = _n_frentes
        plt.figure(figsize=(7, 7))
        for frente in range(1, n_frentes + 1):
            index = self.fronts_dic[str(frente)]
            df = self.df_scores.loc[index]
            sns.lineplot(df[self.methods[0]], df[self.methods[1]])
        pareto_points = self.pareto_front(self.df_scores[self.methods].to_numpy())
        sns.scatterplot(self.df_scores[self.methods[0]], self.df_scores[self.methods[1]], 
                        hue = self.df_scores['Activity'], s=75, palette = ["#50DAB4", "#FA5F50"])
        plt.show()

SyntaxError: invalid syntax (<ipython-input-7-a6681bf5d33d>, line 1)

***
# Empieza el análisis
## Rankings de los ligandos de la mejor conformación

In [ ]:
# Plot del ranking
import matplotlib.pyplot as  plt
import seaborn as sns
%matplotlib inline
sns.set( context = 'talk', style = 'white')

data = df_conf_2vv9.drop(['Activity'], axis = 1).melt()
data['Activity'] = np.tile(is_active, 5)

cmp_ranking = sns.color_palette(["#50DAB4", "#FA5F50"])

fig, axes = plt.subplots()
fig.set_size_inches(15, 10)
sns.swarmplot(x = "variable", y = "value", data= data, ax = axes, alpha=0.8, 
              hue = "Activity", s = 8, palette = cmp_ranking)
axes.set_title('Rankings de los programas de docking')
axes.grid(linestyle='--', linewidth='0.8')
plt.xlabel("Programas/scores de docking")
plt.ylabel("Score del programa")
plt.show()

> ## Nota:  
Los ligandos en los rankings de Vina y Vinardo se ven más apilados porque sus scores (energía de interacción) sólo tienen un cifra decimal mientras que AD4 reporta sus valores con dos cifras decimales.

***
# Frente de pareto de los ligandos con la conf. 2vv9

## Ejemplos por pares de scorings
Como ejemplo de los frentes generados vamos a visualizar por pares de scorings antes de generar frentes con más de un método de docking/scoring.

#### NOTA:
Si el parámetro `just_actives` está como `True`, sólo se calculan los fentes hasta el punto en el que el siguiente frente ya no tiene al menos un ligando activo en él.

### Primer ejemplo (ad4_LC vs vina):

In [ ]:
metodos = ['ad4_LC', 'vina']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.plot_pareto_front(just_actives = True)
a.get_num_of_actives_per_front()

### Todos los posibles frentes
Con el parámetro `just_actives` desactivado, todos los posibles frentes son dibujados (independientemente de si tienen o no un ligando activo en él) hasta que ya no puede encontrarse un frente no dominado con el resto de los puntos.

In [ ]:
metodos = ['ad4_LC', 'vina']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.plot_pareto_front(just_actives = False)
a.get_num_of_actives_per_front()

### Ejemplo 2 (vrd_8x vs vrd_16x)

In [ ]:
metodos = ['vrd_8x', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.plot_pareto_front(just_actives = False)
a.get_num_of_actives_per_front()

### Tercer Ejemplo (vina vs vrd_16x)

In [ ]:
# Vina vs vinardo_16x
metodos = ['vina', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.plot_pareto_front(just_actives = True)
a.get_num_of_actives_per_front()

# Frente combinando más de dos scores

## Los 5 scores (5 objetivos)

In [ ]:
metodos = ['ad4_LE', 'ad4_LC', 'vina', 'vrd_8x', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin ad4_LE 

In [ ]:
metodos = ['ad4_LC', 'vina', 'vrd_8x', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin ad4_LC

In [ ]:
metodos = ['ad4_LE', 'vina', 'vrd_8x', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin Vina

In [ ]:
metodos = ['ad4_LE', 'ad4_LC', 'vrd_8x', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin vinardo_8x

In [ ]:
metodos = ['ad4_LE', 'ad4_LC', 'vina',  'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin vinardo_16x

In [ ]:
metodos = ['ad4_LE', 'ad4_LC', 'vina', 'vrd_8x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin vinardos

In [ ]:
metodos = ['ad4_LE', 'ad4_LC', 'vina']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()

## Sin ad4s

In [ ]:
metodos = ['vina', 'vrd_8x', 'vrd_16x']
a = Pareto(df_conf_2vv9, metodos, minimize = True)
a.get_all_fronts(just_actives = False)
a.get_num_of_actives_per_front()